In [1]:
q="jobname:*r7725.*"

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [3]:
ind="jobs2016"

ss = {
  "size": 0,
  "aggs": {
    "2": {
      "terms": {
        "field": "jobstatus", "size": 5,
        "order": { "_count": "desc" }
      },
      "aggs": {
        "3": { "avg": { "field": "wall_time"  }   },
        "4": { "avg": { "field": "nevents"  } },
        "5": { "avg": {   "field": "cpuconsumptiontime"  }  },
        "6": { "avg": { "field": "corecount" }  },
        "7": { "avg": { "field": "inputfilebytes" }  },
        "8": { "sum": {  "field": "inputfilebytes" }  }
      }
    }
  },
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

In [4]:
res= es.search( index=ind, body=ss, analyze_wildcard=True)

# print(res)
for r in res['aggregations']['2']['buckets']:
    cores=r['6']['value']
    wall=r['3']['value']
    events=r['4']['value']
    cpu=r['5']['value']
    
    if events<1: continue
    secPerEvent=wall/events*cores
    cpueff=cpu/(wall*cores)
    print (r['key'], "\tjobs:", r['doc_count'],"\tevents:", events, '\t avg.cores:', cores)
    print ("wall time:", wall, "\twalltime per event per core:", secPerEvent, "[s/ev/core]" ) 
    print ("cpu time :", cpu,  "\tcputtime per event:", cpu/events ,"\tcpu eff.:", cpueff ) 
    print ('--------------------------------------------------------')


finished 	jobs: 676069 	events: 1998.2958884374227 	 avg.cores: 7.998250168253053
wall time: 19782.197021309956 	walltime per event per core: 79.17894520507201 [s/ev/core]
cpu time : 98756.55787944721 	cputtime per event: 49.42038786691914 	cpu eff.: 0.624160725290306
--------------------------------------------------------
failed 	jobs: 190587 	events: 751.3281178674306 	 avg.cores: 7.999816356834412
wall time: 14727.106974767428 	walltime per event per core: 156.80785593383388 [s/ev/core]
cpu time : 32839.82310440901 	cputtime per event: 43.70902981459226 	cpu eff.: 0.2787426022394922
--------------------------------------------------------
cancelled 	jobs: 2509 	events: 802.3116779593463 	 avg.cores: 8.0
wall time: 36404.279792746114 	walltime per event per core: 362.99389170392453 [s/ev/core]
cpu time : 111054.80749302512 	cputtime per event: 138.4185355191257 	cpu eff.: 0.3813246963175529
--------------------------------------------------------


In [5]:
ss = {
  "fields": ["wall_time","jobstatus"],
  "query": {
    "filtered": {
      "query": {  "query_string": { "analyze_wildcard": True, "lowercase_expanded_terms": False,  "query": q }  }
    }
  }
}

scroll = scan(es, query=ss, index=ind, scroll='10s')
fi=[]
fa=[]
for res in scroll:
    r=res['fields']
    if r['jobstatus'][0]=='finished':
        fi.append(r['wall_time'][0]/3600)
    elif r['jobstatus'][0]=='failed':
        fa.append(r['wall_time'][0]/3600)


ScanError: Scroll request has failed on 2 shards out of 510.

In [ ]:
plt.hist(fi,bins=30, log=True, alpha=0.5, label='finished')
plt.hist(fa,bins=30, log=True, alpha=0.5, label='failed')
plt.legend(loc='upper right')
plt.show()